In [53]:
import numpy as np
import pandas as pd
import ms3 as ms

In [329]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.options.mode.chained_assignment = None

Importing development version of dimcat

In [54]:
import dimcat as dc

In [296]:
# this takes some time because it parses the original data, not the preprocessed tsv files
corpus = dc.Corpus()
corpus.load("../../ABC", parse_tsv=False, parse_scores=True) # make sure to parse directly from MuseScore files
corpus.data

INCOMPLETE_MC_WRONGLY_COMPLETED_WARNING (3, 221) ms3.Parse.ABC.n04op18-4_01.mscx -- /opt/homebrew/lib/python3.10/site-packages/ms3-0.5.3.post0.dev224+ga917290-py3.10.egg/ms3/bs4_measures.py (line 707) make_offset_col():
	The incomplete MC 221 (timesig 1, act_dur 3/4) is completed by 1 incorrect duration (expected: 1/4):
	{79: Fraction(1, 1)}
INCOMPLETE_MC_WRONGLY_COMPLETED_WARNING (3, 67) ms3.Parse.ABC.n13op130_03.mscx -- /opt/homebrew/lib/python3.10/site-packages/ms3-0.5.3.post0.dev224+ga917290-py3.10.egg/ms3/bs4_measures.py (line 707) make_offset_col():
	The incomplete MC 67 (timesig 1, act_dur 5/8) is completed by 1 incorrect duration (expected: 3/8):
	{68: Fraction(1, 1)}
INCOMPLETE_MC_WRONGLY_COMPLETED_WARNING (3, 40) ms3.Parse.ABC.n12op127_02.mscx -- /opt/homebrew/lib/python3.10/site-packages/ms3-0.5.3.post0.dev224+ga917290-py3.10.egg/ms3/bs4_measures.py (line 707) make_offset_col():
	The incomplete MC 40 (timesig 3/2, act_dur 7/8) is completed by 1 incorrect duration (expected: 

282 files.
KEY -> EXTENSIONS
-----------------
ABC -> {'.mscx': 71, '.tsv': 211}

All 71 MSCX files have been parsed.

They all have annotations attached.
KEY -> ANNOTATION LAYERS
------------------------
ABC -> staff  voice  harmony_layer  color  
    -> 4      1      1 (dcml)       default    27625
    ->               1              default        1
    -> 1      1      1 (dcml)       default      731

1/211 tabular files have been parsed, 0 of them as Annotations object(s).

In [297]:
labels = corpus.get_facet("expanded")
labels

DCML_HARMONY_SYNTAX_WARNING (15,) ms3.Parse.ABC.n06op18-6_04.mscx -- /opt/homebrew/lib/python3.10/site-packages/ms3-0.5.3.post0.dev224+ga917290-py3.10.egg/ms3/annotations.py (line 350) expand_dcml():
	Score contains 1 labels that don't (and 464 that do) match the DCML standard:
	    mc  mn      label harmony_layer
	22  18  18  V7[viio64             1
DCML_HARMONY_INCOMPLETE_PEDAL_COLUMN_ERROR (13,) ms3.Parse.ABC.n06op18-6_04.mscx -- /opt/homebrew/lib/python3.10/site-packages/ms3-0.5.3.post0.dev224+ga917290-py3.10.egg/ms3/expand_dcml.py (line 155) expand_labels():
	propagate_pedal() failed with
	1 organ points started, 2 ended:
	     mc pedal   mc pedalend
	0   106     I   20        ]
	1  <NA>   NaN  108        ]
DCML_HARMONY_INCOMPLETE_PEDAL_COLUMN_ERROR (13,) ms3.Parse.ABC.n10op74_04.mscx -- /opt/homebrew/lib/python3.10/site-packages/ms3-0.5.3.post0.dev224+ga917290-py3.10.egg/ms3/expand_dcml.py (line 155) expand_labels():
	propagate_pedal() failed with
	5 organ points started, 6 ended

quarterbeats  duration_qb   mc   mn  \
corpus fname        interval                                               
ABC    n01op18-1_01 [0.0, 3.0)                  0          3.0    1    1   
                    [3.0, 6.0)                  3          3.0    2    2   
                    [6.0, 9.0)                  6          3.0    3    3   
                    [9.0, 15.0)                 9          6.0    4    4   
                    [15.0, 18.0)               15          3.0    6    6   
...                                           ...          ...  ...  ...   
       n16op135_04  [1140.0, 1141.0)         1140          1.0  281  280   
                    [1141.0, 1142.0)         1141          1.0  281  280   
                    [1142.0, 1144.0)         1142          2.0  282  281   
                    [1144.0, 1146.0)         1144          2.0  282  281   
                    [1146.0, 1150.0)         1146          4.0  283  282   

                                     mc_onset mn_onset timesig  staff  voice  \
corpus fname        interval                                                   
ABC    n01op18-1_01 [0.0, 3.0)              0        0     3/4      4      1   
                    [3.0, 6.0)              0        0     3/4      4      1   
                    [6.0, 9.0)              0        0     3/4      4      1   
                    [9.0, 15.0)             0        0     3/4      4      1   
                    [15.0, 18.0)            0        0     3/4      4      1   
...                                       ...      ...     ...    ...    ...   
       n16op135_04  [1140.0, 1141.0)      1/2      1/2     4/4      4      1   
                    [1141.0, 1142.0)      3/4      3/4     4/4      4      1   
                    [1142.0, 1144.0)        0        0     4/4      4      1   
                    [1144.0, 1146.0)      1/2      1/2     4/4      4      1   
                    [1146.0, 1150.0)        0        0     4/4      4      1   

                                      volta  ... phraseend chord_type  \
corpus fname        interval                 ...                        
ABC    n01op18-1_01 [0.0, 3.0)         <NA>  ...       NaN          M   
                    [3.0, 6.0)         <NA>  ...       NaN          M   
                    [6.0, 9.0)         <NA>  ...       NaN          M   
                    [9.0, 15.0)        <NA>  ...       NaN          M   
                    [15.0, 18.0)       <NA>  ...       NaN        Mm7   
...                                     ...  ...       ...        ...   
       n16op135_04  [1140.0, 1141.0)   <NA>  ...       NaN          M   
                    [1141.0, 1142.0)   <NA>  ...       NaN          M   
                    [1142.0, 1144.0)   <NA>  ...       NaN          M   
                    [1144.0, 1146.0)   <NA>  ...       NaN        Mm7   
                    [1146.0, 1150.0)   <NA>  ...        \\          M   

                                     globalkey_is_minor localkey_is_minor  \
corpus fname        interval                                                
ABC    n01op18-1_01 [0.0, 3.0)                    False             False   
                    [3.0, 6.0)                    False             False   
                    [6.0, 9.0)                    False             False   
                    [9.0, 15.0)                   False             False   
                    [15.0, 18.0)                  False             False   
...                                                 ...               ...   
       n16op135_04  [1140.0, 1141.0)               True             False   
                    [1141.0, 1142.0)               True             False   
                    [1142.0, 1144.0)               True             False   
                    [1144.0, 1146.0)               True             False   
                    [1146.0, 1150.0)               True             False   

                                        chord_tones add

In [178]:
# this takes some time
salami_crp = dc.NoteSlicer().process_data(corpus)
salami_notes = salami_crp.get_facet("notes")

In [300]:
# zoom in on the chords in one piece
chords = labels.loc[('ABC', 'n01op18-1_03')]

# Translate labels to absolute pitches


In [355]:
desired_chord_columns = ['chord','pedal','numeral','form','figbass','changes','relativeroot','localkey','globalkey']
chordz = chords.copy()[desired_chord_columns].reset_index()
chordz = chordz.reset_index()

In [71]:
chords.to_csv('chords.csv')

In [373]:
clean_chords = chordz[chordz['chord'] != '@none']
ms.labels2global_tonic(clean_chords, inplace=True)
chordsAbs = clean_chords.copy()[['interval', 'chord']]


In [377]:
chordsAbs.loc[138:143]

,interval,chord
138,"[221.0, 222.0)",viio64
139,"[222.0, 223.0)",V(+2)
141,"[224.0, 225.0)",vii%7
142,"[225.0, 226.0)",I
143,"[226.0, 227.0)",V43


In [378]:
pd.concat([chordsAbs, chordz[chordz['chord'] == '@none']])

,interval,chord,pedal,numeral,form,figbass,changes,relativeroot,localkey,globalkey
0,"[0.0, 1.0)",I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[1.0, 3.0)",I6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[3.0, 4.0)",V(4),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[4.0, 5.0)",V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[5.0, 6.0)",VI43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
199,"[422.0, 423.0)",V(b64),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,"[423.0, 426.0)",V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,"[426.0, 429.0)",V7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,"[429.0, 435.0)",V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [242]:
chordz.loc[0:2]

,chord,pedal,numeral,form,figbass,changes,relativeroot,localkey,globalkey
interval,,,,,,,,,
"[0.0, 1.0)",I,NaN,I,NaN,NaN,NaN,NaN,I,F
"[1.0, 3.0)",I6,NaN,I,NaN,6,NaN,NaN,I,F


In [266]:
chordzz = chordz.reset_index()

In [286]:
chordzz.loc[140:145]

,interval,chord,pedal,numeral,form,figbass,changes,relativeroot,localkey,globalkey
140,"[223.0, 224.0)",@none,NaN,@none,NaN,NaN,NaN,NaN,I,F
141,"[224.0, 225.0)",vii%7,NaN,vii,%,7,NaN,NaN,I,F
142,"[225.0, 226.0)",I,NaN,I,NaN,NaN,NaN,NaN,I,F
143,"[226.0, 227.0)",V43,NaN,V,NaN,43,NaN,NaN,I,F
144,"[227.0, 228.0)",I6,NaN,I,NaN,6,NaN,NaN,I,F
145,"[228.0, 229.0)",ii6,NaN,ii,NaN,6,NaN,NaN,I,F


In [289]:
chordsAbs = ms.labels2global_tonic(chordzz.loc[140:145])

/opt/homebrew/lib/python3.10/site-packages/ms3-0.5.3.post0.dev224+ga917290-py3.10.egg/ms3/transformations.py:548: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, cols['pedal']] = transform(df, rel2abs_key, param_cols)
@none is not a valid scale degree.


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [288]:
chordsAbs

,numeral,form,figbass,changes,globalkey,globalkey_is_minor
141,vii,%,7,NaN,F,False
142,I,NaN,NaN,NaN,F,False
143,V,NaN,43,NaN,F,False
144,I,NaN,6,NaN,F,False
145,ii,NaN,6,NaN,F,False


In [106]:
#chordz['leftInt'] = chordz.index.left

In [165]:
#chordz.set_index('leftInt')

# Process Slices

In [83]:
salamis = salami_notes.loc[("ABC", "n01op18-1_03")]

In [232]:
mini_salamis = salamis[['midi','tpc','tied']]
mini_salamis['tied'] = mini_salamis['tied'].fillna(0).astype('bool')
mini_salamis


/var/folders/zq/32m4qp9s339211sg4wxlr7dr0000gn/T/ipykernel_9452/2943445955.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini_salamis['tied'] = mini_salamis['tied'].fillna(0).astype('bool')


midi  tpc   tied
onset_slice    interval                        
[0.0, 1.0)     [0.0, 1.0)        41   -1  False
               [0.0, 1.0)        57    3  False
               [0.0, 1.0)        60    0   True
               [0.0, 1.0)        65   -1   True
[1.0, 2.0)     [1.0, 2.0)        57    3  False
...                             ...  ...    ...
[432.5, 433.0) [432.5, 433.0)    71    5  False
[433.0, 433.5) [433.0, 433.5)    74    2  False
[433.5, 434.0) [433.5, 434.0)    72    0  False
[434.0, 434.5) [434.0, 434.5)    70   -2  False
[434.5, 435.0) [434.5, 435.0)    67    1  False

[2031 rows x 3 columns]

In [167]:
#merged = pd.merge_asof(chords, mini_salamis, left_on='leftInt', right_on='leftInt')
#merged.iloc[:10]

Assigning each slice a segment id according to the chord.

In [233]:
# Bad practice to use loops with pandas..
dfs = []
for segment, interval in enumerate(chordz.index):
    segMask = mini_salamis.index.get_level_values(0).overlaps(interval)
    slicesInInterval = mini_salamis[segMask]
    slicesInInterval.insert(0,'segment_id',segment)
    dfs.append(slicesInInterval)

In [234]:
segmented_salamis = pd.concat(dfs)

In [235]:
segmented_salamis.insert(0,"onset_slice_start", segmented_salamis.index.get_level_values(0).left)
segmented_salamis.insert(1, "onset_slice_end", segmented_salamis.index.get_level_values(0).right)

In [236]:
segmented_salamis.reset_index()

,onset_slice,interval,onset_slice_start,onset_slice_end,segment_id,midi,tpc,tied
0,"[0.0, 1.0)","[0.0, 1.0)",0.0,1.0,0,41,-1,False
1,"[0.0, 1.0)","[0.0, 1.0)",0.0,1.0,0,57,3,False
2,"[0.0, 1.0)","[0.0, 1.0)",0.0,1.0,0,60,0,True
3,"[0.0, 1.0)","[0.0, 1.0)",0.0,1.0,0,65,-1,True
4,"[1.0, 2.0)","[1.0, 2.0)",1.0,2.0,1,57,3,False
...,...,...,...,...,...,...,...,...
2026,"[432.5, 433.0)","[432.5, 433.0)",432.5,433.0,202,71,5,False
2027,"[433.0, 433.5)","[433.0, 433.5)",433.0,433.5,202,74,2,False
2028,"[433.5, 434.0)","[433.5, 434.0)",433.5,434.0,202,72,0,False
2029,"[434.0, 434.5)","[434.0, 434.5)",434.0,434.5,202,70,-2,False


In [237]:
segmented_salamis.to_csv('salamis.csv')